In [1]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr
from langchain.document_loaders import DirectoryLoader, TextLoader, PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_chroma import Chroma
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_core.callbacks import StdOutCallbackHandler ## By using this line we can see how langchain works

In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-


In [3]:
MODEL = "claude-sonnet-4-20250514"
db_name = "vector_db"

In [4]:
## We can use this code to read pdf files
folders = glob.glob("pdfs/*")
text_loader_kwargs = {'encoding': 'utf-8'}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)

    # Load PDF files
    pdf_loader = DirectoryLoader(
        folder,
        glob="**/*.pdf",
        loader_cls=PyPDFLoader
    )
    pdf_docs = pdf_loader.load()
    for doc in pdf_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [6]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: files


In [7]:
embeddings = OpenAIEmbeddings()

In [8]:
## Delete if already exists
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

In [9]:
vectorstore = Chroma.from_documents(documents = chunks, 
                                    embedding = embeddings, 
                                    persist_directory = db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 136 documents


In [10]:
## Get one vector and find how many dimensions it has
collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 1,536 dimensions


In [11]:
## create a new Chat with OpenAI (model)
llm = ChatAnthropic(
    temperature = 0.7,
    model = MODEL
)

## set up the conversation memory for the chat (Memory)
memory = ConversationBufferMemory(
    memory_key = 'chat_history', 
    return_messages = True)

## the retriever is an abstraction over the VectorStore that will be used during RAG (Retreiver)
retriever = vectorstore.as_retriever()
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm = llm, 
    retriever = retriever, 
    memory = memory)

/var/folders/27/gbz_5drj4l13l6x8mnk54j1w0000gn/T/ipykernel_15739/2220161630.py:8: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [12]:
def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [13]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
